In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
# Import BatchNormalization and L2 regularization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import pandas as pd

# Import necessary components for preprocessing
from pathlib import Path
from PIL import Image, ImageEnhance
from typing import Tuple, List

# --- Preprocessing Configuration (Copied from image_preprocessor.py) ---
# NOTE: The path should ideally be configured to a relative path or an environment variable,
# but keeping the absolute path as per the original for functionality.
DATA_ROOT = r'C:\Users\acking\Desktop\project\DeepCrack-An-SDNET2018-Implementation\raw data\Walls - Copy'
TARGET_SIZE: Tuple[int, int] = (256, 256)
BRIGHTNESS_FACTOR: float = 1.5
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp']
# --- Model Configuration ---
L2_REG_STRENGTH = 1e-4 # L2 Regularization strength
MAX_EPOCHS = 50       # Increased maximum epochs for better convergence, used with Early Stopping

def load_and_preprocess_data(root_dir: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Loads images and applies the requested preprocessing steps:
    Brightness enhancement, resizing, and normalization.
    (Function remains unchanged as preprocessing logic seems sound)
    """
    root_path = Path(root_dir)
    data: List[np.ndarray] = []
    labels: List[int] = []

    categories = {'cracked': 1, 'non-cracked': 0}
    print(f"Starting data preprocessing...")

    for category, label in categories.items():
        folder_path = root_path / category
        if not folder_path.is_dir():
            print(f"Warning: Category folder not found: {folder_path}. Skipping.")
            continue

        print(f"\nProcessing category '{category}' (Label {label})...")
        count = 0

        for file_path in folder_path.rglob('*'):
            if file_path.suffix.lower() not in IMAGE_EXTENSIONS:
                continue

            try:
                # Use 'RGB' for consistency, even if some are grayscale
                with Image.open(file_path).convert('RGB') as img:
                    img_resized = img.resize(TARGET_SIZE)

                    # 2. Increase Brightness (Enhancement)
                    enhancer = ImageEnhance.Brightness(img_resized)
                    img_enhanced = enhancer.enhance(BRIGHTNESS_FACTOR)

                    # 3. Normalize (Convert to array and scale)
                    img_array = np.array(img_enhanced, dtype=np.float32)
                    normalized_array = img_array / 255.0

                    data.append(normalized_array)
                    labels.append(label)
                    count += 1

            except Exception as e:
                # print(f"Could not process {file_path}: {e}") # Debugging line
                pass

        print(f"Successfully processed {count} images for '{category}'.")

    if not data:
        print("\nFATAL: No valid images were processed. Cannot build model.")
        return np.array([]), np.array([])

    X = np.array(data)
    y = np.array(labels)
    return X, y

def build_cnn_model(input_shape: Tuple[int, int, int]) -> Sequential:
    """
    Defines an improved Convolutional Neural Network (CNN) architecture with:
    - Batch Normalization for stability.
    - L2 Regularization for overfitting control.
    - Increased layer depth (filters).
    """
    print("Building IMPROVED CNN model...")
    model = Sequential([
        # Block 1: Larger initial kernel, L2 Regularization
        Conv2D(32, (5, 5), activation='relu', input_shape=input_shape, padding='same',
               kernel_regularizer=l2(L2_REG_STRENGTH)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.2), # Reduced initial dropout, as BN also regularizes

        # Block 2: Increase filters
        Conv2D(64, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(L2_REG_STRENGTH)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        # Block 3: Further increase filters
        Conv2D(128, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(L2_REG_STRENGTH)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),

        # Block 4 (New) - Deeper network often helps
        Conv2D(256, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(L2_REG_STRENGTH)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),

        # Classification Head
        Flatten(),
        Dense(256, activation='relu', kernel_regularizer=l2(L2_REG_STRENGTH)),
        BatchNormalization(),
        Dropout(0.5), # Standard heavier dropout before final layer
        Dense(1, activation='sigmoid') # Binary classification
    ])

    # Compile the model with Adam optimizer
    model.compile(
        optimizer=Adam(learning_rate=1e-3), # Explicitly set a standard learning rate
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# The following utility functions are kept as they are well-written and necessary for reporting.
# save_error_metrics_csv and generate_plots

def save_error_metrics_csv(y_test: np.ndarray, y_pred_class: np.ndarray, file_path: Path):
    """
    Calculates Confusion Matrix and Classification Report and saves them to a CSV file.
    """
    # Calculate the Confusion Matrix (the 'matrix of errors')
    cm = confusion_matrix(y_test, y_pred_class)
    cm_df = pd.DataFrame(cm,
                         index=['Actual Non-Cracked', 'Actual Cracked'],
                         columns=['Predicted Non-Cracked', 'Predicted Cracked'])

    # Calculate the Classification Report (the 'error metrix')
    report = classification_report(y_test, y_pred_class, target_names=['Non-Cracked (0)', 'Cracked (1)'], output_dict=True)
    report_df = pd.DataFrame(report).transpose()

    # Combine data into a single CSV
    with open(file_path, 'w') as f:
        f.write("--- Confusion Matrix ---\n")
        cm_df.to_csv(f)
        f.write("\n--- Classification Report (Error Metrics) ---\n")
        report_df.to_csv(f)

    print(f"\nMetrics and Confusion Matrix saved to: {file_path.name}")


def generate_plots(history: tf.keras.callbacks.History, y_test: np.ndarray, y_pred_proba: np.ndarray):
    """
    Generates and saves five charts visualizing training history and model performance.
    (No changes needed to the plotting logic)
    """
    history_dict = history.history
    epochs = range(1, len(history_dict['loss']) + 1)

    # --- Chart 1: Accuracy History ---
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, history_dict['accuracy'], 'bo', label='Training Acc')
    plt.plot(epochs, history_dict['val_accuracy'], 'b', label='Validation Acc')
    plt.title('1. Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('accuracy_history.png')
    plt.close()

    # --- Chart 2: Loss History ---
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, history_dict['loss'], 'ro', label='Training Loss')
    plt.plot(epochs, history_dict['val_loss'], 'r', label='Validation Loss')
    plt.title('2. Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss_history.png')
    plt.close()

    # --- Chart 3: Confusion Matrix ---
    cm = confusion_matrix(y_test, (y_pred_proba > 0.5).astype("int32"))
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('3. Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['Non-Cracked', 'Cracked'])
    plt.yticks(tick_marks, ['Non-Cracked', 'Cracked'])

    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

    # --- Chart 4: ROC Curve ---
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title('4. Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig('roc_curve.png')
    plt.close()

    # --- Chart 5: Precision-Recall Curve ---
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='purple', lw=2, label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('5. Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.grid(True)
    plt.savefig('precision_recall_curve.png')
    plt.close()

    print("\nFive visualization charts saved as PNG files (1-5.png) in the current directory.")


def main():
    """Main function to load data, train, evaluate, and generate reports."""

    # --- 1. Load and Preprocess Data ---
    if not Path(DATA_ROOT).is_dir():
        print(f"ERROR: The main data root directory does not exist at: {DATA_ROOT}")
        print("Please verify the path and ensure it contains 'cracked' and 'non-cracked' folders.")
        return

    X, y = load_and_preprocess_data(DATA_ROOT)

    if X.size == 0:
        return

    input_shape = X.shape[1:]

    # --- 2. Split Data ---
    print("\nSplitting data into training (80%) and testing (20%) sets...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"Training images: {X_train.shape[0]}, Testing images: {X_test.shape[0]}")

    # --- 3. Build Model (Improved architecture) ---
    model = build_cnn_model(input_shape)
    model.summary()

    # --- 4. Define Callbacks for Training Optimization ---
    callbacks = [
        # Stop training if validation loss doesn't improve for 5 epochs
        EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
        # Reduce learning rate if val_loss plateaus, often helps convergence
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-6)
    ]

    # --- 5. Train Model ---
    print("\nStarting model training with Early Stopping and Learning Rate Reduction...")
    history = model.fit(
        X_train, y_train,
        epochs=MAX_EPOCHS,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=callbacks, # Apply the defined callbacks
        verbose=1
    )

    # --- 6. Evaluate Model & Generate Reports ---
    print("\n=======================================================")
    print("             MODEL EVALUATION ON TEST DATA")
    print("=======================================================")
    # Reload best weights if EarlyStopping was triggered
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    print(f"Final Test Loss: {loss:.4f}")
    print(f"Final Test Accuracy: {accuracy*100:.2f}%")

    # Get probability predictions and convert them to class predictions (0 or 1)
    y_pred_proba = model.predict(X_test).ravel()
    y_pred_class = (y_pred_proba > 0.5).astype("int32")

    # Generate and save the CSV of metrics and Confusion Matrix
    save_error_metrics_csv(y_test, y_pred_class, Path('model_metrics.csv'))

    # Generate and save the five requested charts
    generate_plots(history, y_test, y_pred_proba)

if __name__ == "__main__":
    # Suppress TensorFlow warnings/info messages (optional)
    tf.get_logger().setLevel('ERROR')
    main()


Starting data preprocessing...

Processing category 'cracked' (Label 1)...
Successfully processed 846 images for 'cracked'.

Processing category 'non-cracked' (Label 0)...
Successfully processed 873 images for 'non-cracked'.

Splitting data into training (80%) and testing (20%) sets...
Training images: 1375, Testing images: 344
Building IMPROVED CNN model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      2432      
                                                                 
 batch_normalization (Batch  (None, 256, 256, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                        

C:\Users\acking\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\acking\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\acking\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


Metrics and Confusion Matrix saved to: model_metrics.csv

Five visualization charts saved as PNG files (1-5.png) in the current directory.
